In [68]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Conv2D , MaxPooling2D , AveragePooling2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator , load_img , img_to_array
from keras import backend

In [86]:
home_path = "chest_xray"
train_path = "chest_xray/train"
val_path = "chest_xray/val"
test_path = "chest_xray/test"

image_w = 128
image_h = 128
stride = (3,3)
pool = (2,2)
batch_size = 60
if backend.image_data_format() == "channels_first":
    input_shape = (3 , image_w , image_h)
else:
    input_shape = (image_w , image_h , 3)

In [134]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.25,
    width_shift_range=(-100,100),
    height_shift_range=(-100,100),
    preprocessing_function=pre_process
)

In [88]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(image_w , image_h),
    batch_size=batch_size,
    class_mode="binary"
)
train_generator.class_indices

Found 5216 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}

In [73]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(image_w , image_h),
    batch_size=batch_size,
    class_mode="binary"
)
val_generator.class_indices

Found 16 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}

In [74]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(image_w , image_h),
    batch_size=batch_size,
    class_mode="binary"
)
test_generator.class_indices

Found 624 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}

In [133]:
def pre_process(img):
    sss = np.ones_like(img)
    sss[:] = 255
    return sss

In [75]:
model = Sequential()

model.add( Conv2D(32 , stride , padding="same" , activation="relu" , input_shape=input_shape) )
model.add( MaxPooling2D(pool_size=pool) )

model.add( Dropout(.5) )

model.add( Conv2D(64 , stride , padding="same" , activation="relu") )
model.add( MaxPooling2D(pool_size=pool) )

model.add( Dropout(.5) )

model.add( Flatten() )

model.add( Dense(16 , activation="relu") )

model.add( Dense(1 , activation="sigmoid") )

model.compile(optimizer="rmsprop" , loss="binary_crossentropy" , metrics=["accuracy"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)            

In [76]:
model.fit_generator(
    train_generator,
    epochs=2,
    validation_data=val_generator ,
)

Epoch 1/2
87/87 [==============================] - 98s 1s/step - loss: 0.7065 - accuracy: 0.7993 - val_loss: 0.5710 - val_accuracy: 0.6250
Epoch 2/2
87/87 [==============================] - 104s 1s/step - loss: 0.2063 - accuracy: 0.9224 - val_loss: 0.6493 - val_accuracy: 0.6250


In [77]:
loss, acc = model.evaluate_generator(test_generator , verbose=1)
loss , acc

11/11 [==============================] - 7s 631ms/step - loss: 0.7673 - accuracy: 0.7035


(0.7673215866088867, 0.7035256624221802)

In [135]:
i = 0
imgg = load_img("chest_xray/train/NORMAL/IM-0115-0001.jpeg")
x = img_to_array(imgg)
x = x.reshape((1,) + (x.shape))
gen = train_datagen.flow(x , batch_size = 1 , save_to_dir=home_path+"/saved" , save_prefix="AUG")
for batch in gen:
    i += 1
    if i >= 5: # save 20 images
        break